In [ ]:
import numpy as np
import scipy

def moment_legendre(n):
    return 1/(1+np.arange(n))

def jacobi_a_legendre(n):
    return 0.5*np.ones( (n,) )

def jacobi_b_legendre(n):
    k = 1.0*np.arange(1, n+1)
    b = 1/( 4*(4-k**(-2)) )
    #b = np.array( [1] + list( b ) )
    return np.sqrt(b)

def moment_laguerre(n):
    k = 1.0*np.arange(0, n)
    k[0] = 1.0
    return np.cumprod(k)

def jacobi_a_laguerre(n):
    k = 1.0*np.arange(1, n+1)
    return 2*k-1

def jacobi_b_laguerre(n):
    k = 1.0*np.arange(1, n+1)
    return k


In [ ]:
oprl_setup = 'legendre'

if oprl_setup == 'legendre':
    compute_moments  = moment_legendre
    compute_jacobi_a = jacobi_a_legendre
    compute_jacobi_b = jacobi_b_legendre
    scipy_quadrature = scipy.special.roots_legendre
elif oprl_setup == 'laguerre':
    compute_moments  = moment_laguerre
    compute_jacobi_a = jacobi_a_laguerre
    compute_jacobi_b = jacobi_b_laguerre
    scipy_quadrature = scipy.special.roots_laguerre

# I. Direct approach: Cholesky

In [ ]:
import freeDeconvolution

In [ ]:
# Form moment matrix
n = 2
moments_count = 2*n+1
mom_array = compute_moments( moments_count + 1)
print( f'''Computed moments from 0 to 2n+1={len(mom_array)-1}''')

# Compute
#jacobi_a, jacobi_b = freeDeconvolution.oprl.jacobi_from_moments( mom_array )
jacobi_a, jacobi_b = freeDeconvolution.jacobi_from_moments( mom_array )

# Print Jacobi coefficients
print( "Computed Jacobi coefficients:")
print( "b: ", jacobi_b )
print( "a: ", jacobi_a )
print( "")

# Given n moments
jacobi_dim = len(jacobi_a)
print( "Theoretical Jacobi coefficients:")
print( "b: ", compute_jacobi_b(jacobi_dim-1) )
print( "a: ", compute_jacobi_a(jacobi_dim) )
print( "")

In [ ]:
# Quadrature measure from our code
print("Quadrature measure")
support, weights = freeDeconvolution.quadrature_from_jacobi( jacobi_a, jacobi_b)
print( support )
print( weights )
# Compute weights via vandermonde (unstable)
# vandermonde = np.vander( eigen ).T
# weights = np.linalg.solve( vandermonde, mom_array[0:mom_count] )
# print( "Raw Weights: ")
# print( weights)
print("")

# Quadrature from scipy
print( "Ground truth")
import scipy
points, weights, mass = scipy_quadrature( len(support), mu=True)
if scipy_quadrature == scipy.special.roots_legendre:
    points = (points+1)/2
print( points )
weights = weights/weights.sum()
print( weights )

# II. Iterative approach: Three term recurrence

NOT WORKING YET

In this approach, we avoid computing moments at all cost! Perhaps this has better stability

For the monic polynomials $P_n$:
$$ X P_n = P_{n+1} + a_n P_n + b_{n-1}^2 P_{n-1} $$

From that one deduces (after some work) that:
$$ \| P_n \| = b_1 \dots b_{n-1}, $$
which yields the recurrence for orthonormal polynomials $p_n$:
$$ X p_n = b_n p_{n+1} + a_n p_n + b_{n-1} p_{n-1} $$

In [ ]:
def polynomial_eval( coeffs, points):
    m = len(coeffs)
    powers = np.arange( 0, m, 1)
    vander = points[..., None]**powers[None, ...]
    return np.dot(vander, coeffs)

# Test
points = np.arange( 0, 5)
coefficients = [1, 0]
print( polynomial_eval(coefficients, points) )
coefficients = [1, 1]
print( polynomial_eval(coefficients, points) )


In [ ]:
# Compute OPRL
polynomials = []
norms = []

# First in the iteration: P_0 = 1
polynomials.append( [1] )
norms.append( 1 )

# Second in the iteration: P_1 = X-m_1
m1 = np.real( cauchy_integral_g_deconv( z_array ) )
polynomials.append( [-m1, 1] )
P1 = np.array( polynomials[-1] )
values_P1 = polynomial_eval(P1, z_array)
norm2_P1 = np.real( cauchy_integral_g_deconv( values_P1*values_P1 ) )
norm_P1  = np.sqrt(norm2_P1)
assert( norm2_P1 > 0)
norms.append( norm_P1 )

# Apply three term recurrence
OPRL_order = 10
jacobi_a = [ m1 ]
jacobi_b = [ ]
for n in np.arange(1, OPRL_order, 1):
    Pn  = polynomials[-1] + [0.0]
    XPn = [0.0] + polynomials[-1]
    Qn  = polynomials[-2] + [0.0, 0.0] # Short for P_{n-1}
    Pn, XPn, Qn = [ np.array(x) for x in [Pn, XPn, Qn] ]
    # Values along path
    values_Pn  = polynomial_eval(  Pn, z_array)
    values_XPn = z_array*values_Pn
    values_Qn  = polynomial_eval(  Qn, z_array)
    # Norms
    norm_Pn = norms[-1]
    norm_Qn = norms[-2]
    # Compte a_n and b_{n-1}
    a  = np.real( cauchy_integral_g_deconv( values_XPn*values_Pn ) )/( norm_Pn)
    b2 = np.real( cauchy_integral_g_deconv( values_XPn*values_Qn ) )/( norm_Qn)
    print( "n: ", n+1)
    print( "a: ", a)
    print( "b2: ", b2)
    print( "")
    if b2 < 0:
        print( "Defaulting at n =", n+1)
        print( "")
        OPRL_order = n
        #jacobi_a.append( a )
        break
    #
    b = np.sqrt(b2)
    jacobi_a.append( a )
    jacobi_b.append( b )
    norms.append( norm_Pn*b )
    # Three term recurrence
    new_P = XPn - a*Pn - b*b*Qn
    polynomials.append( list(new_P) )
# end for
print( "Jacobi coefficients:")
print( jacobi_a )
print( jacobi_b )
print( "")
print( "Norms:")
print( norms )